### Fair Bank Account with Monitors (Java)

A bank account is shared by several people (processes). Each person may deposit or withdraw funds from the account. The current balance in the account is the sum of all deposits to date minus the sum of all withdrawals. The balance must never become negative. A process that withdraws an amount that would make the balance negative has to wait until the balance is large enough.

Implement a monitor in Java to solve the problem. The monitor has two procedures (methods): `deposit(amount)` and `withdraw(amount)`, for positive integer `amount`. If the balance is `0` and processes are requesting to withdraw 300 and 100 and depositing 100, 120, and 180, in that order, an acceptable solution is that the process requesting 100 will be served right after another process deposits 100, even if the request for 300 came before the request for 100. That is, requests should be served as soon as a request can be served. Do not use Java library classes for `Account` (there is no need; the solution is short), though you may use whatever you like for testing. State the monitor invariant!

The implementation below is *instrumented* with statements that log the execution of `deposit()` and `withdraw()`. This is used for testing that all deposits and withdrawals occur and occur in a valid order. For example, the log could be `D100D180D120W100W300`.

In [ ]:
%%writefile Account.java
import java.util.regex.Pattern;
class Account {
    # YOUR CODE HERE
    raise NotImplementedError()
    synchronized void deposit(int amount) {
        # YOUR CODE HERE
        raise NotImplementedError()
        TestAccount.log += "D" + amount;
    }
    synchronized void withdraw(int amount) {
        # YOUR CODE HERE
        raise NotImplementedError()
        TestAccount.log += "W" + amount;
    }
}
class Depositer extends Thread {
    Account a;
    int amount;
    Depositer(Account a, int amount) {
        this.a = a; this.amount = amount;
    }
    public void run() {
        a.deposit(amount);
    }
}
class Withdrawer extends Thread {
    Account a;
    int amount;
    Withdrawer(Account a, int amount) {
        this.a = a; this.amount = amount;
    }
    public void run() {
        a.withdraw(amount);
    }
}
class TestAccount {
    static String log = "";
    public static void main(String[] args) {
        Account a = new Account();
        Withdrawer w0 = new Withdrawer(a, 300); w0.start();
        try {Thread.sleep(10);} catch (Exception x) {}
        Withdrawer w1 = new Withdrawer(a, 100); w1.start();
        Depositer d0 = new Depositer(a, 100); d0.start();
        try {Thread.sleep(10);} catch (Exception x) {}
        Depositer d1 = new Depositer(a, 120); d1.start();
        Depositer d2 = new Depositer(a, 180); d2.start();
        try {w0.join(); w1.join(); d0.join(); d1.join(); d2.join();
        } catch(Exception e) {}
        System.out.println(log);
        assert Pattern.matches(".*W300.*", log) : "300 not withdrawn";
        assert Pattern.matches(".*W100.*", log) : "100 not withdrawn";
        assert Pattern.matches(".*D100.*", log) : "100 not deposited";
        assert Pattern.matches(".*D120.*", log) : "120 not deposited";
        assert Pattern.matches(".*D180.*", log) : "180 not deposited";
        assert Pattern.matches(".*D....*W100.*", log) : "100 withdrawn before any deposit";
        assert Pattern.matches(".*(D120.*D180|D180.*D120).*W300.*", log) :
            "300 withdrawn before both 120 and 180 deposited";
    }
}

In [ ]:
!javac Account.java

In [ ]:
!java -ea TestAccount

As the problem is stated above, there is no fairness among processes withdrawing an amount. In particular, a process withdrawing a large amount that may only be satisfied by several deposits will starve if smaller withdrawals are served immediately. Now, modify the implementation such that withdrawals follow the first-come-first-serve discipline. State the monitor invariant!

Testing is now modified to ensure that 300 is attempted to be withdrawn before 100. A valid log is `D100D180D120W300W100`.

_Hint:_ Use the idea of the ticket algorithm. Remember that each process calling `withdraw` and `deposit` has its stack and, therefore, its copy of local variables. The solution requires about ten lines of code.

In [ ]:
%%writefile Account.java
import java.util.regex.Pattern;
class Account {
    int balance = 0; // balance >= 0
    int number = 0, next = 0; // number >= next >= 0
    synchronized void deposit(int amount) {
        # YOUR CODE HERE
        raise NotImplementedError()
        TestAccount.log += "D" + amount;
    }
    synchronized void withdraw(int amount) {
        # YOUR CODE HERE
        raise NotImplementedError()
        TestAccount.log += "W" + amount;
    }
}
class Depositer extends Thread {
    Account a;
    int amount;
    Depositer(Account a, int amount) {
        this.a = a; this.amount = amount;
    }
    public void run() {
        a.deposit(amount);
    }
}
class Withdrawer extends Thread {
    Account a;
    int amount;
    Withdrawer(Account a, int amount) {
        this.a = a; this.amount = amount;
    }
    public void run() {
        a.withdraw(amount);
    }
}
class TestAccount {
    static String log = "";
    public static void main(String[] args) {
        Account a = new Account();
        Withdrawer w0 = new Withdrawer(a, 300); w0.start();
        // wait 10 ms to ensure that w0 tries to withdraw before w1
        try {Thread.sleep(10);} catch (Exception x) {}
        Withdrawer w1 = new Withdrawer(a, 100); w1.start();
        Depositer d0 = new Depositer(a, 100); d0.start();
        try {Thread.sleep(10);} catch (Exception x) {}
        Depositer d1 = new Depositer(a, 120); d1.start();
        Depositer d2 = new Depositer(a, 180); d2.start();
        try {w0.join(); w1.join(); d0.join(); d1.join(); d2.join();
        } catch(Exception e) {}
        System.out.println(log);
        assert Pattern.matches(".*W300.*W100.*", log) : "300 must be withdrawn before 100 withdrawn";
        assert Pattern.matches(".*D120.*W300.*", log) : "120 must be deposited before 300 withdrawn";
        assert Pattern.matches(".*D180.*W300.*", log) : "180 must be deposited before 300 withdrawn";
        assert Pattern.matches(".*D100.*W100.*", log) : "120 must be deposited before 100 withdrawn";
    }
}

In [ ]:
!javac Account.java

In [ ]:
!java -ea TestAccount